## Instanzen und Erweiterungen

In diesem Kapitel werden mehrere Quantenvariationsalgorithmen behandelt, darunter:

- [Variationeller Quanteneigenlöser (VQE)](https://arxiv.org/abs/1304.3061)
- [Subraumsuche VQE (SSVQE)](https://arxiv.org/abs/1810.09434)
- [Variationelle Quantendeflation (VQD)](https://arxiv.org/abs/1805.08138)
- [Quanten-Sampling-Regression (QSR)](https://arxiv.org/pdf/2012.02338)

Durch die Verwendung dieser Algorithmen lernen wir verschiedene Designideen kennen, die in benutzerdefinierte Variationsalgorithmen integriert werden können, z. B. Gewichtungen, Strafen, Überabtastung und Unterabtastung. Wir ermutigen Sie, mit diesen Konzepten zu experimentieren und Ihre Erkenntnisse mit der Community zu teilen.

## Variationeller Quanteneigenlöser (VQE)

[VQE](https://arxiv.org/abs/1304.3061) ist einer der am weitesten verbreiteten Variationsquantenalgorithmen und stellt eine Vorlage dar, auf der andere Algorithmen aufbauen können.

![VQE](images/instances_VQE.png)

Das Layout von VQE ist einfach:

- Bereiten Sie die Referenzoperatoren $U_R$ vor
    - Wir beginnen mit dem Zustand $|0\rangle$ und gehen zum Referenzzustand $|\rho\rangle$ über
- Wenden Sie die Variationsform $U_V(\vec\theta_{i,j})$ an, um einen Ansatz $U_A(\vec\theta_{i,j})$ zu erstellen
    - Wir gehen vom Zustand $|\rho\rangle$ zu $U_V(\vec\theta_{i,j})|\rho\rangle = |\psi(\vec\theta_{i,j})\rangle$
- Bootstrap bei $i=0$, wenn wir ein ähnliches Problem haben (normalerweise durch klassische Simulation oder Stichprobe gefunden)
    - Jeder Optimierer wird anders gebootet, was zu einem anfänglichen Satz von Parametervektoren $\Theta_0 := \{ {\vec\theta_{0,j} | führt j \in \mathcal{J}_\text{opt}^0} \}$ (z. B. von einem Anfangspunkt $\vec\theta_0$).
- Bewerten Sie die Kostenfunktion $C(\vec\theta_{i,j}) := \langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle$ für alle vorbereiteten Zustände auf einem Quantencomputer.
- Verwenden Sie einen klassischen Optimierer, um den nächsten Parametersatz $\Theta_{i+1}$ auszuwählen.
- Wiederholen Sie den Vorgang, bis die Konvergenz erreicht ist.

Dies ist eine einfache klassische Optimierungsschleife, in der wir die Kostenfunktion auswerten. Einige Optimierer erfordern möglicherweise mehrere Auswertungen, um einen Gradienten zu berechnen, die nächste Iteration zu bestimmen oder die Konvergenz zu bewerten.

Hier ist das Beispiel für die folgende Observable:

$$
\hat{O}_1 = 2 II - 2 XX + 3 YY - 3 ZZ,
$$

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
import numpy as np

# Add your token below
service = QiskitRuntimeService(
    channel="ibm_quantum",
)

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
    reference_circuit = QuantumCircuit(2)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        2,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values

Mit dieser Kostenfunktion können wir optimale Parameter berechnen

In [ ]:
from qiskit.algorithms.optimizers import COBYLA

initial_theta = np.ones(8)
optimizer = COBYLA()

optimizer_result = optimizer.minimize(fun=cost_function_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

Schließlich können wir unsere optimalen Parameter verwenden, um unsere minimalen Eigenwerte zu berechnen:

In [ ]:
observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
reference_circuit = QuantumCircuit(2)
reference_circuit.x(0)

variational_form = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
ansatz = reference_circuit.compose(variational_form)
solution = ansatz.bind_parameters(optimal_parameters)

backend = service.backend("ibmq_qasm_simulator")
estimator = Estimator(session=backend)
job = estimator.run(solution, observable)
values = job.result().values

experimental_min_eigenvalue = values[0]
print(experimental_min_eigenvalue)

## Subraumsuche VQE (SSVQE)

[SSVQE](https://arxiv.org/abs/1810.09434) ist eine Variante von VQE, die es ermöglicht, die ersten $k$ Eigenwerte einer Observablen $\hat{H}$ mit den Eigenwerten {$\lambda_0, \lambda_1,...,\lambda_{N-1}$} zu erhalten, wobei $N\geq k$. Ohne Beschränkung der Allgemeinheit gehen wir davon aus, dass $\lambda_0&lt;\lambda_1&lt;...&lt;\lambda_{N-1}$. SSQVE führt eine neue Idee ein, indem Gewichtungen hinzugefügt werden, um die Optimierung für den Term mit der größten Gewichtung zu priorisieren.

![SSVQE](images/instances_SSVQE.png)

Um diesen Algorithmus zu implementieren, benötigen wir $k$ zueinander orthogonale Referenzzustände `{latex} \{ |\rho_j\rangle \}_{j=0}^{k-1}` , also $\langle \rho_j | \rho_l \rangle = \delta_{jl}$ für $j,l&lt;k$. Diese Zustände können mit Pauli-Operatoren konstruiert werden. Die Kostenfunktion dieses Algorithmus lautet dann:

$$
\begin{aligned}
C(\vec{\theta}) 

& := \sum_{j=0}^{k-1} w_j \langle \rho_j | U_{V}^{\dagger}(\vec{\theta})\hat{H} U_{V}(\vec{\theta})|\rho_j \rangle \\[1mm]

& := \sum_{j=0}^{k-1} w_j \langle \psi_{j}(\vec{\theta}) | \hat{H} | \psi_{j}(\vec{\theta}) \rangle \\[1mm]

\end{aligned}
$$

wobei $w_j$ eine beliebige positive Zahl ist, so dass, wenn $j&lt;l&lt;k$, dann $w_j&gt;w_l$ ist und $U_V(\vec{\theta})$ die benutzerdefinierte Variationsform ist.

Der SSVQE-Algorithmus basiert auf der Tatsache, dass Eigenzustände, die unterschiedlichen Eigenwerten entsprechen, zueinander orthogonal sind. Insbesondere kann das innere Produkt von $U_V(\vec{\theta})|\rho_j\rangle$ und $U_V(\vec{\theta})|\rho_l\rangle$ ausgedrückt werden als:

$$
\begin{aligned}
\langle \rho_j | U_{V}^{\dagger}(\vec{\theta})U_{V}(\vec{\theta})|\rho_l \rangle

& = \langle \rho_j | I |\rho_l \rangle \\[1mm]

& = \langle \rho_j | \rho_l \rangle \\[1mm]

& = \delta_{jl}

\end{aligned}
$$

Die erste Gleichung gilt, weil $U_{V}(\vec{\theta})$ ein Quantenoperator und daher einheitlich ist. Die letzte Gleichheit gilt aufgrund der Orthogonalität der Referenzzustände $|\rho_j\rangle$. Die Tatsache, dass die Orthogonalität durch einheitliche Transformationen erhalten bleibt, hängt eng mit dem Prinzip der Informationserhaltung zusammen, wie es in der Quanteninformationswissenschaft zum Ausdruck kommt. Aus dieser Sicht stellen nicht-unitäre Transformationen Prozesse dar, bei denen Informationen entweder verloren gehen oder eingefügt werden.

Gewichte $w_j$ tragen dazu bei, dass alle Zustände Eigenzustände sind. Wenn die Gewichte ausreichend unterschiedlich sind, erhält der Term mit dem größten Gewicht (d. h. $w_0$) bei der Optimierung Vorrang vor den anderen. Infolgedessen wird der resultierende Zustand $U_{V}(\vec{\theta})|\rho_0 \rangle$ zum Eigenzustand, der $\lambda_0$ entspricht. `{latex} \{ U_{V}(\vec{\theta})|\rho_j\rangle \}_{j=0}^{k-1}` , also im Unterraum enthalten, der den Eigenwerten {$\lambda_1,...,\lambda_{N-1}$} entspricht.

Wenn man das gleiche Argument auf den Rest der Terme anwendet, wäre die nächste Priorität dann der Term mit der Gewichtung $w_1$, also wäre $U_{V}(\vec{\theta})|\rho_1 \rangle$ der entsprechende Eigenzustand zu $\lambda_1$, und die anderen Terme wären im Eigenraum von {$\lambda_2,...,\lambda_{N-1}$} enthalten.

Durch induktives Denken folgern wir, dass $U_{V}(\vec{\theta})|\rho_j \rangle$ ein ungefährer Eigenzustand von $\lambda_j$ für $0\leq j &lt; k$ sein wird.

SSVQE's können wie folgt zusammengefasst werden:

- Bereiten Sie mehrere Referenzzustände vor, indem Sie ein einheitliches U_R auf k verschiedene rechnerische Basiszustände anwenden
    - Dieser Algorithmus erfordert die Verwendung von $k$ zueinander orthogonalen Referenzzuständen `{latex} \{ |\rho_j\rangle \}_{j=0}^{k-1}` , so dass $\langle \rho_j | \rho_l \rangle = \delta_{jl}$ für $j,l&lt;k$.
- Wenden Sie die Variationsform $U_V(\vec\theta_{i,j})$ auf jeden Referenzzustand an, was den folgenden Ansatz $U_A(\vec\theta_{i,j})$ ergibt.
- Bootstrap bei $i=0$, wenn ein ähnliches Problem verfügbar ist (normalerweise durch klassische Simulation oder Stichprobe gefunden).
- Bewerten Sie die Kostenfunktion $C(\vec\theta_{i,j}) := \sum_{j=0}^{k-1} w_j \langle \psi_{j}(\vec{\theta}) | \hat{H} | \psi_{j}(\vec{\theta}) \rangle$ für alle vorbereiteten Zustände auf einem Quantencomputer.
    - Dies kann in die Berechnung des Erwartungswerts für ein beobachtbares $\langle \psi_{j}(\vec{\theta}) | unterteilt werden \hat{H} | \psi_{j}(\vec{\theta}) \rangle$ und Multiplikation dieses Ergebnisses mit $w_j$.
    - Anschließend gibt die Kostenfunktion die Summe aller gewichteten Erwartungswerte zurück.
- Verwenden Sie einen klassischen Optimierer, um den nächsten Parametersatz $\Theta_{i+1}$ zu bestimmen.
- Wiederholen Sie die oben genannten Schritte, bis die Konvergenz erreicht ist.

Sie werden die Kostenfunktion von SSVQE in der Bewertung rekonstruieren!

## Variationelle Quantendeflation (VQD)

[VQD](https://arxiv.org/abs/1805.08138) ist eine iterative Methode, die VQE erweitert, um die ersten $k$ Eigenwerte einer Observablen $\hat{H}$ mit den Eigenwerten {$\lambda_0, \lambda_1,...,\lambda_{N-1}$} zu erhalten, wobei $N\geq k$, statt nur das erste. Für den Rest dieses Abschnitts gehen wir ohne Beschränkung der Allgemeinheit davon aus, dass $\lambda_0\leq\lambda_1\leq...\leq\lambda_{N-1}$. VQD führt den Begriff der Strafkosten ein, um den Optimierungsprozess zu steuern.

![VQD](images/instances_VQD.png)

VQD führt einen Strafterm ein, der als $\beta$ bezeichnet wird, um den Beitrag jedes Überlappungsterms zu den Kosten auszugleichen. Dieser Strafterm dient dazu, den Optimierungsprozess zu bestrafen, wenn die Orthogonalität nicht erreicht wird. Wir legen diese Einschränkung fest, weil die Eigenzustände einer Observablen oder eines hermiteschen Operators, die verschiedenen Eigenwerten entsprechen, immer zueinander orthogonal sind oder im Fall von Entartung oder wiederholten Eigenwerten so gemacht werden können. Indem wir also die Orthogonalität mit dem Eigenzustand erzwingen, der $\lambda_0$ entspricht, optimieren wir effektiv über den Unterraum, der den restlichen Eigenwerten {$\lambda_1, \lambda_2,..., \lambda_{N-1}$ entspricht }. Hier ist $\lambda_1$ der niedrigste Eigenwert unter den übrigen Eigenwerten und daher kann die optimale Lösung des neuen Problems mithilfe des Variationssatzes erhalten werden.

Die allgemeine Idee hinter VQD besteht darin, VQE wie üblich zu verwenden, um den niedrigsten Eigenwert $\lambda_0 := C_0(\vec\theta^0) \equiv C_\text{VQE}(\vec\theta^0)$ zusammen mit dem zu erhalten entsprechender (ungefährer) Eigenzustand $|\psi(\vec{\theta^0})\rangle$ für einen optimalen Parametervektor $\vec{\theta^0}$. Um dann den nächsten Eigenwert $\lambda_1 &gt; \lambda_0$ zu erhalten, muss die Kostenfunktion $C_0(\vec{\theta}) := \langle \psi(\vec{\theta}) | minimiert werden \hat{H} | \psi(\vec{\theta})\rangle$, wir optimieren:

$$
C_1(\vec{\theta}) := 
C_0(\vec{\theta})+ \beta_0 |\langle \psi(\vec{\theta})| \psi(\vec{\theta^0})\rangle  |^2 
$$

Der positive Wert $\beta_0$ sollte idealerweise größer als $\lambda_1-\lambda_0$ sein.

Dies führt eine neue Kostenfunktion ein, die als eingeschränktes Problem betrachtet werden kann, wobei wir $C_\text{VQE}(\vec{\theta}) = \langle \psi(\vec{\theta}) | minimieren \hat{H} | \psi(\vec{\theta})\rangle$ unterliegt der Einschränkung, dass der Zustand orthogonal zum zuvor erhaltenen $|\psi(\vec{\theta^0})\rangle$ sein muss, mit $\beta_0$ fungiert als Strafterm, wenn die Einschränkung nicht erfüllt ist.

Alternativ kann dieses neue Problem so interpretiert werden, dass VQE auf dem neuen Observable ausgeführt wird:

$$
\hat{H_1} := \hat{H} + \beta_0 |\psi(\vec{\theta^0})\rangle \langle \psi(\vec{\theta^0})|
\quad \Rightarrow \quad 
C_1(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H_1} | \psi(\vec{\theta})\rangle,
$$

Unter der Annahme, dass die Lösung des neuen Problems $|\psi(\vec{\theta^1})\rangle$ ist, sollte der erwartete Wert von $\hat{H}$ (nicht $\hat{H_1}$) sein $ \langle \psi(\vec{\theta^1}) | \hat{H} | \psi(\vec{\theta^1})\rangle = \lambda_1$.

Um den dritten Eigenwert $\lambda_2$ zu erhalten, lautet die zu optimierende Kostenfunktion:

$$
C_2(\vec{\theta}) := 
C_1(\vec{\theta}) + \beta_1 |\langle \psi(\vec{\theta})| \psi(\vec{\theta^1})\rangle  |^2 
$$

wobei $\beta_1$ eine positive Konstante ist, die groß genug ist, um die Orthogonalität des Lösungszustands sowohl zu $|\psi(\vec{\theta^0})\rangle$ als auch zu $|\psi(\vec{\theta^1) zu erzwingen })\rangle$. Dadurch werden Staaten im Suchraum bestraft, die diese Anforderung nicht erfüllen, wodurch der Suchraum effektiv eingeschränkt wird. Daher sollte die optimale Lösung des neuen Problems der Eigenzustand sein, der $\lambda_2$ entspricht.

Wie der vorherige Fall kann auch dieses neue Problem als VQE mit der Observablen interpretiert werden:

$$
\hat{H_2} := \hat{H_1} + \beta_1 |\psi(\vec{\theta^1})\rangle \langle \psi(\vec{\theta^1})|
\quad \Rightarrow \quad 
C_2(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H_2} | \psi(\vec{\theta})\rangle.
$$

Wenn die Lösung für dieses neue Problem $|\psi(\vec{\theta^2})\rangle$ lautet, sollte der erwartete Wert von $\hat{H}$ (nicht $\hat{H_2}$) $ sein \langle \psi(\vec{\theta^2}) | \hat{H} | \psi(\vec{\theta^2})\rangle = \lambda_2$.

Um den $k$-ten Eigenwert $\lambda_{k-1}$ zu erhalten, würden Sie analog die Kostenfunktion minimieren:

$$
C_{k-1}(\vec{\theta}) := 
C_{k-2}(\vec{\theta}) + \beta_{k-2} |\langle \psi(\vec{\theta})| \psi(\vec{\theta^{k-2}})\rangle  |^2,
$$

Denken Sie daran, dass wir $\vec{\theta^j}$ so definiert haben, dass $\langle \psi(\vec{\theta^j}) | \hat{H} | \psi(\vec{\theta^j})\rangle = \lambda_j, \forall j&lt;k$. Dieses Problem entspricht der Minimierung von $C(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle$, aber mit der Einschränkung, dass der Zustand orthogonal zu $|\psi(\vec{\theta^j})\rangle sein muss; \forall j \in {0, \cdots, k-1}$, wodurch der Suchraum auf den Unterraum beschränkt wird, der den Eigenwerten {$\lambda_{k-1},\cdots,\lambda_{N-1}$ entspricht }.

Dieses Problem entspricht einem VQE mit der Observablen:

$$
\hat{H}_{k-1} := 
\hat{H}_{k-2} + \beta_{k-2} |\psi(\vec{\theta^{k-2}})\rangle \langle \psi(\vec{\theta^{k-2}})|
\quad \Rightarrow \quad 
C_{k-1}(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H}_{k-1} | \psi(\vec{\theta})\rangle,
$$

Wie Sie dem Prozess entnehmen können, benötigen Sie zum Erhalten des $k$-ten Eigenwerts die (ungefähren) Eigenzustände der vorherigen $k-1$-Eigenwerte, sodass Sie VQE insgesamt $k$-mal ausführen müssten. Daher lautet die Kostenfunktion von VQD wie folgt:

$$
C_k(\vec{\theta}) =
\langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle +
\sum_{j=0}^{k-1}\beta_j |\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle |^2
$$

Das Layout von VQD lässt sich wie folgt zusammenfassen:

- Bereiten Sie einen Referenzoperator $U_R$ vor
- Wenden Sie die Variationsform $U_V(\vec\theta_{i,j})$ auf den Referenzzustand an und erstellen Sie den folgenden Ansatz $U_A(\vec\theta_{i,j})$
- Bootstrap bei $i=0$, wenn wir ein ähnliches Problem haben (normalerweise durch klassische Simulation oder Stichprobe gefunden).
- Bewerten Sie die Kostenfunktion $C_k(\vec{\theta})$, was die Berechnung von $k$ angeregten Zuständen und einem Array von $\beta$s umfasst, die die Überlappungsstrafe für jeden Überlappungsterm definieren.
    - Berechnen Sie den Erwartungswert für eine Observable $\langle \psi_{j}(\vec{\theta}) | \hat{H} | \psi_{j}(\vec{\theta}) \rangle$ für jedes $k$
    - Berechnen Sie die Strafe $\sum_{j=0}^{k-1}\beta_j |\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle |^2$.
    - Die Kostenfunktion sollte dann die Summe dieser beiden Terme zurückgeben
- Verwenden Sie einen klassischen Optimierer, um den nächsten Parametersatz $\Theta_{i+1}$ auszuwählen.
- Wiederholen Sie diesen Vorgang, bis die Konvergenz erreicht ist.

## Quanten-Sampling-Regression (QSR)

Eines der Hauptprobleme bei VQE sind die mehrfachen Aufrufe an einen Quantencomputer, die erforderlich sind, um die Parameter für jeden Schritt zu erhalten, einschließlich $k$, $k-1$ usw. Dies ist besonders problematisch, wenn der Zugriff auf Quantengeräte in der Warteschlange steht . Während eine [`Session`](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/run_session.html) zum Gruppieren mehrerer iterativer Aufrufe verwendet werden kann, besteht ein alternativer Ansatz in der Verwendung von Stichproben. Durch den Einsatz klassischerer Ressourcen können wir den gesamten Optimierungsprozess in einem einzigen Anruf abschließen. Hier kommt [die Quantum Sampling Regression](https://arxiv.org/pdf/2012.02338) ins Spiel. Da der Zugang zu Quantencomputern immer noch ein Gut mit geringem Angebot und hoher Nachfrage ist, halten wir diesen Kompromiss für viele aktuelle Studien für möglich und praktisch. Dieser Ansatz nutzt alle verfügbaren klassischen Fähigkeiten und erfasst gleichzeitig viele der inneren Abläufe und intrinsischen Eigenschaften von Quantenberechnungen, die in der Simulation nicht auftauchen.

![QSR](images/instances_QSR.png)

Die Idee hinter QSR ist, dass die Kostenfunktion $C(\theta) := \langle \psi(\theta) | \hat{H} | \psi(\theta)\rangle$ kann auf folgende Weise als Fourier-Reihe ausgedrückt werden:

$$
\begin{aligned}
C(\vec{\theta}) 

& := \langle \psi(\theta) | \hat{H} | \psi(\theta)\rangle \\[1mm]

& := a_0 + \sum_{k=1}^S[a_k\cos(k\theta)+ b_k\sin(k\theta)] \\[1mm]

\end{aligned}
$$

Abhängig von der Periodizität und Bandbreite der ursprünglichen Funktion kann die Menge $S$ endlich oder unendlich sein. Für die Zwecke dieser Diskussion gehen wir davon aus, dass es unendlich ist. Der nächste Schritt besteht darin, die Kostenfunktion $C(\theta)$ mehrmals abzutasten, um die Fourier-Koeffizienten ${a_0, a_k, b_k}_{k=1}^S$ zu erhalten. Da wir insbesondere $2S+1$ Unbekannte haben, müssen wir die Kostenfunktion $2S+1$ mal abtasten.

Wenn wir dann die Kostenfunktion für $2S+1$-Parameterwerte {$\theta_1,...,\theta_{2S+1}$} abtasten, können wir das folgende System erhalten:

$$
\begin{pmatrix} 1 & \cos(\theta_1) & \sin(\theta_1) & \cos(2\theta_1) & ... & \sin(S\theta_1) \\
1 & \cos(\theta_2) & \sin(\theta_2) & \cos(2\theta_2) & \cdots & \sin(S\theta_2)\\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots\\ 
1 & \cos(\theta_{2S+1}) & \sin(\theta_{2S+1}) & \cos(2\theta_{2S+1}) & \cdots & \sin(S\theta_{2S+1})
\end{pmatrix} \begin{pmatrix} a_0 \\ a_1 \\ b_1 \\ a_2 \\ \vdots \\ b_S \end{pmatrix} = \begin{pmatrix} C(\theta_1) \\ C(\theta_2) \\ \vdots \\ C(\theta_{2S+1}) \end{pmatrix},
$$

dass wir umschreiben werden als

$$
Fa=c.
$$

In der Praxis ist dieses System im Allgemeinen nicht konsistent, da die Kostenfunktionswerte $c$ nicht exakt sind. Daher ist es normalerweise eine gute Idee, sie zu normalisieren, indem man sie links mit $F^\dagger$ multipliziert, was zu Folgendem führt:

$$
F^\dagger Fa = F^\dagger c.
$$

Dieses neue System ist immer konsistent und seine Lösung ist eine Lösung der kleinsten Quadrate für das ursprüngliche Problem. Wenn wir $k$-Parameter anstelle von nur einem haben und jeder Parameter $\theta^i$ sein eigenes $S_i$ für $i \in {1,...,k}$ hat, dann ist die Gesamtzahl der erforderlichen Stichproben Ist:

$$
T=\prod_{i=1}^k(2S_i+1)\leq \prod_{i=1}^k(2S_{max}+1) = (2S_{max}+1)^n,
$$

wobei $S_{\max} = \max_i(S_i)$. Darüber hinaus eröffnet die Anpassung von $S_{\max}$ als einstellbarem Parameter (anstatt ihn abzuleiten) neue Möglichkeiten, wie zum Beispiel:

- **Oversampling** : zur Verbesserung der Genauigkeit.
- **Unterabtastung** : Steigerung der Leistung durch Reduzierung des Laufzeit-Overheads oder Eliminierung lokaler Minima.

Das Layout von QSR lässt sich wie folgt zusammenfassen:

- Bereiten Sie die Referenzoperatoren $U_R$ vor
    - Wir gehen vom Zustand $|0\rangle$ zum Referenzzustand $|\rho\rangle$ über
- Wenden Sie die Variationsform $U_V(\vec\theta_{i,j})$ an, um einen Ansatz $U_A(\vec\theta_{i,j})$ zu erstellen
    - Bestimmen Sie die Bandbreite, die jedem Parameter im Ansatz zugeordnet ist. Eine Obergrenze ist ausreichend.
- Bootstrap bei $i=0$, wenn wir ein ähnliches Problem haben (normalerweise durch klassische Simulation oder Stichprobe gefunden)
- Probieren Sie die Kostenfunktion $C(\vec\theta) := a_0 + \sum_{k=1}^S[a_k\cos(k\theta)+ b_k\sin(k\theta)]$ mindestens $T$ aus mal
    - $T=\prod_{i=1}^k(2S_i+1)\leq \prod_{i=1}^k(2S_{max}+1) = (2S_{max}+1)^n$
    - Entscheiden Sie, ob eine Über- oder Unterabtastung durchgeführt werden soll, um Geschwindigkeit und Genauigkeit auszugleichen, indem Sie $T$ anpassen.
- Berechnen Sie die Fourier-Koeffizienten aus den Stichproben (dh lösen Sie das normalisierte lineare Gleichungssystem).
- Lösen Sie das globale Minimum der resultierenden Regressionsfunktion auf einer klassischen Maschine auf.

In dieser Lektion haben Sie die verschiedenen verfügbaren Variationsinstanzen kennengelernt:

- Hauptgestaltung
- Einführung von Gewichten und Strafen zur Anpassung einer Kostenfunktion
- Untersuchung von Unterabtastung und Überabtastung, um einen Kompromiss zwischen Geschwindigkeit und Genauigkeit zu finden

Diese Ideen können angepasst werden, um einen benutzerdefinierten Variationsalgorithmus zu bilden, der zu Ihrem Problem passt. Wir ermutigen Sie, Ihre Ergebnisse mit der Community zu teilen. In der nächsten Lektion erfahren Sie, wie Sie einen Variationsalgorithmus zum Lösen einer Anwendung verwenden.